In [97]:
!pip -q install crewai  crewai-tools==0.28.8 --ignore-installed blinker

ERROR: Could not find a version that satisfies the requirement crewai-tools==0.28.8 (from versions: 0.0.1, 0.0.2, 0.0.3, 0.0.5, 0.0.6, 0.0.7, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.14, 0.0.15, 0.0.16, 0.1.0, 0.1.1, 0.1.2, 0.1.3, 0.1.4, 0.1.5, 0.1.6, 0.1.7, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.2.5, 0.2.6, 0.3.0, 0.4.0, 0.4.1, 0.4.3, 0.4.4, 0.4.5, 0.4.6, 0.4.7, 0.4.8, 0.4.25, 0.4.26, 0.8.0, 0.8.3, 0.12.0, 0.12.1, 0.13.0, 0.13.1, 0.13.2, 0.13.4, 0.14.0, 0.17.0, 0.25.0, 0.25.1, 0.25.2, 0.25.3, 0.25.4, 0.25.5, 0.25.6, 0.25.7, 0.25.8, 0.32.0, 0.32.1, 0.33.0, 0.36.0, 0.37.0, 0.38.0)
ERROR: No matching distribution found for crewai-tools==0.28.8


In [81]:
%%writefile agents.py
from crewai import Agent
from tools import yt_tool
from crewai import LLM

## Create a senior blog content researcher 
from dotenv import load_dotenv
load_dotenv()
import os 
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_MODEL_NAME']="gpt-4.5-preview-2025-02-27"
#os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')
llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.7
)
blog_researcher= Agent(
 role="Blog Researcher from Youtube vids",
 goal="Get the relevant video content for the topic {topic} from YT",
 verbose=True,
 memory=True,
 backstory=(
  "This agent is an expert in understanding videos in Data Science and ML"   
 ),
 tool=[yt_tool],
 llm=llm,
 allow_delegation=True
       
)


## create a write agent 
blog_writer= Agent(
 role="Blog writer",
 goal="Narate compiling tech stories about the video {topic} from YT channel",
 verbose=True,
 memory=True,
 backstory=(
  "Your an expert in crafting blogs"   
 ),
 tool=[yt_tool],
 llm=llm,
 allow_delegation=False
       
)


Overwriting agents.py


In [98]:
%%writefile tools.py

from crewai import Agent, Task, Crew
from crewai_tools import YoutubeChannelSearchTool

yt_tool = YoutubeChannelSearchTool(youtube_channel_handle="@krishnaik06")



Overwriting tools.py


In [99]:
%%writefile tasks.py

from crewai import Task
from tools import yt_tool
from agents import blog_writer, blog_researcher

#research task 
research_task=Task(
    description=("identify the video {topic}"
    "get detailed description of the video"
                ),
    expected_output="A comprehensive 3 paragraph long report based on the {topic} of video",
    tools=[yt_tool],
    agent=blog_researcher
)


writing_task=Task(
    description=("identify the video {topic}"
    "get detailed description of the video"
                ),
    expected_output="Summarize the info from the youtube channel on the topic {topic} and create content for blog",
    tools=[yt_tool],
    agent=blog_writer,
    async_execution=False,
    output_file='new-blog-post.md'# as markdown
)

Overwriting tasks.py


In [100]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [101]:
%%writefile crew.py
from crewai import Crew, Process
from agents import blog_writer, blog_researcher
from tasks import research_task,writing_task

crew=Crew(
    agents=[blog_researcher, blog_writer],
    tasks=[research_task, writing_task],
    process=Process.sequential,
    memory=True,
    cache=True,
    share_crew=True,
    max_rpm=100
)

#task execution with enchanced feedback

result=crew.kickoff(inputs={'topic':'AI VS ML vs Data Science'})
#result = crew.kickoff(inputs={"youtube_channel_handle": "@exampleChannel"})

print(result)


Overwriting crew.py


In [ ]:
!python crew.py